In [ ]:
import jams
import os
from tqdm import tqdm

In [ ]:
def round_to_decimals(value, decimals):
    """Round a value to a specific number of decimal places."""
    return round(value, decimals)


def lab_to_jams(lab_file, jams_file):
    # Create a new JAMS object
    jam = jams.JAMS()

    # Create a new annotation for chords
    ann = jams.Annotation(namespace="chord")

    # Read the .lab file and calculate the duration
    max_end_time = 0.0
    with open(lab_file, "r") as f:
        for line in f:
            # Split the line into start time, end time, and chord label
            line = line.strip()
            if len(line) == 0:
                continue

            start, end, chord = line.split()

            # Convert times to float
            start = float(start)
            end = float(end)

            # Determine the number of decimal places in the input
            start_str = f"{start}"  # Convert to string
            if "." in start_str:
                decimals = len(start_str.split(".")[1])  # Count decimal places
            else:
                decimals = 0

            # Round the start and end times to the same number of decimal places
            start = round_to_decimals(start, decimals)
            end = round_to_decimals(end, decimals)

            # Update the maximum end time
            if end > max_end_time:
                max_end_time = end

            # Add the chord observation to the annotation
            ann.append(
                time=start,
                duration=round_to_decimals(end - start, decimals),
                value=chord,
            )

    # Add the annotation to the JAMS object
    jam.annotations.append(ann)

    # Set the file metadata duration (required by JAMS schema)
    jam.file_metadata.duration = round_to_decimals(max_end_time, decimals)

    # Save the JAMS object to a .jams file
    jam.save(jams_file)

In [ ]:
datasets_path = (
    "ADDRESS_TO_YOUR_DATASET"  # Replace with the path to your dataset
)
for root, dirs, files in os.walk(datasets_path):
    for file in tqdm(files):
        if file.endswith(".lab"):
            labs_file_path = os.path.join(root, file)
            jams_file_path = labs_file_path.replace(".lab", ".jams")
            try:
                lab_to_jams(labs_file_path, jams_file_path)
            except Exception as e:
                print(f"Error processing {labs_file_path}")
                continue